## Process single cell morphology features for CellProfiler readouts - Comparisons

Compare the output of `pycytominer` and `pycytominer-transform` using NF1_SchwannCells_data output.

In [1]:
import os
import pathlib
import warnings

import pandas as pd
from pycytominer import normalize
from pycytominer.cyto_utils import cells, output
from pycytominer_transform import config, convert

# ignore warnings
warnings.filterwarnings("ignore")

In [2]:
# Set file and directory constants
cp_dir = "../CellProfiler_pipelines"
output_dir = "data"

# gather sorted list of source data filepaths
single_cell_filepaths = sorted(
    list(
        pathlib.Path(f"{cp_dir}/Analysis_Output/").glob(
            "**/NF1_data_allcp_plate*.sqlite"
        )
    )
)

# prepare platemap filepath
platemap_file = f"{cp_dir}/Metadata/platemap_NF1_CP.csv"

# prepare output data filepaths
sc_output_file = pathlib.Path(f"{output_dir}/nf1_sc_cellprofiler.csv.gz")
sc_norm_output_file = pathlib.Path(f"{output_dir}/nf1_sc_norm_cellprofiler.csv.gz")

print("single_cell_filepaths:")
single_cell_filepaths

single_cell_filepaths:


[PosixPath('../CellProfiler_pipelines/Analysis_Output/Plate1_Output/NF1_data_allcp_plate1.sqlite'),
 PosixPath('../CellProfiler_pipelines/Analysis_Output/Plate2_Output/NF1_data_allcp_plate2.sqlite')]

In [3]:
# Define custom linking columns between compartments
linking_cols = {
    "Per_Cytoplasm": {
        "Per_Cells": "Cytoplasm_Parent_Cells",
        "Per_Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Per_Cells": {"Per_Cytoplasm": "Cells_Number_Object_Number"},
    "Per_Nuclei": {"Per_Cytoplasm": "Nuclei_Number_Object_Number"},
}

In [4]:
# Load platemap file
platemap_df = pd.read_csv(platemap_file)
platemap_df

,WellRow,WellCol,well_position,gene_name,genotype
0,C,6,C6,NF1,WT
1,C,7,C7,NF1,Null
2,D,6,D6,NF1,WT
3,D,7,D7,NF1,Null
4,E,6,E6,NF1,WT
5,E,7,E7,NF1,Null
6,F,6,F6,NF1,WT
7,F,7,F7,NF1,Null


In [5]:
# show file to be processed below
single_cell_filepath_one = single_cell_filepaths[0]
single_cell_filepath_one

PosixPath('../CellProfiler_pipelines/Analysis_Output/Plate1_Output/NF1_data_allcp_plate1.sqlite')

In [6]:
%%time
# pycytominer
# perform merge single cells without annotation
# and export to parquet format, re-reading the result
# from the parquet file for precision in comparison
pycytominer_sc_df_without_annotation = pd.read_parquet(
    # path is the output filename of SingleCells.merge_single_cells
    path=cells.SingleCells(
        sql_file=f"sqlite:///{single_cell_filepath_one}",
        compartments=["Per_Cells", "Per_Cytoplasm", "Per_Nuclei"],
        compartment_linking_cols=linking_cols,
        image_table_name="Per_Image",
        strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
        merge_cols=["ImageNumber"],
        image_cols="ImageNumber",
        load_image_data=True,
        # perform merge_single_cells without annotation
        # and receive parquet filepath
    ).merge_single_cells(
        sc_output_file="pycytominer_singlecells_merge.parquet",
        output_type="parquet",
    )
)

CPU times: user 458 ms, sys: 40 ms, total: 498 ms
Wall time: 456 ms


In [7]:
pycytominer_sc_df_without_annotation.info()
pycytominer_sc_df_without_annotation.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 0 to 241
Columns: 1205 entries, Metadata_ImageNumber to Nuclei_Texture_Variance_RFP_3_03_256
dtypes: float64(1184), int64(19), object(2)
memory usage: 2.2+ MB


,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Well,Cytoplasm_Number_Object_Number,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,1,001,C6,1,32065.0,95944.0,1018.0,382.0,750.0,24.0,...,1393.410857,1331.583275,653.826838,618.063979,606.832257,590.114791,147.195839,144.355017,148.179465,148.875403
1,1,001,C6,2,14466.0,58032.0,688.0,337.0,440.0,103.0,...,1369.498111,1276.305513,332.941295,317.567450,321.873215,292.116754,60.632767,61.876198,65.202076,60.022847
2,1,001,C6,3,41368.0,139598.0,1201.0,503.0,888.0,57.0,...,1338.091947,1299.373271,432.829034,398.306003,401.091835,358.849840,74.837374,71.033793,80.523205,80.845266
3,1,001,C6,4,14564.0,52624.0,377.0,470.0,169.0,217.0,...,899.439956,874.837386,211.898029,189.348918,186.333300,188.292692,113.059608,113.194846,110.997393,109.834390
4,1,001,C6,5,27417.0,84084.0,760.0,550.0,487.0,242.0,...,1231.630414,1218.998954,306.139730,295.581509,310.469726,287.788390,496.084704,502.046808,490.259298,491.171009


In [9]:
# show configuration preset SQL for pycytominer-transform
print(config["cellprofiler_sqlite"]["CONFIG_JOINS"])


            WITH Per_Image_Filtered AS (
                SELECT
                    Metadata_ImageNumber,
                    Image_Metadata_Well,
                    Image_Metadata_Plate
                FROM
                    read_parquet('per_image.parquet')
                )
            SELECT
                *
            FROM
                Per_Image_Filtered AS per_image
            LEFT JOIN read_parquet('per_cytoplasm.parquet') AS per_cytoplasm ON
                per_cytoplasm.Metadata_ImageNumber = per_image.Metadata_ImageNumber
            LEFT JOIN read_parquet('per_cells.parquet') AS per_cells ON
                per_cells.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_cells.Cells_Number_Object_Number = per_cytoplasm.Cytoplasm_Parent_Cells
            LEFT JOIN read_parquet('per_nuclei.parquet') AS per_nuclei ON
                per_nuclei.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_nuclei.Nuclei_Num

In [10]:
# make a copy of default configuration presets from pycytominer-transform
modified_config = config.copy()
# replace Cytoplasm_Parent_OrigNuclei value in modified config
modified_config["cellprofiler_sqlite"]["CONFIG_JOINS"] = modified_config[
    "cellprofiler_sqlite"
]["CONFIG_JOINS"].replace(
    "per_cytoplasm.Cytoplasm_Parent_OrigNuclei", "per_cytoplasm.Cytoplasm_Parent_Nuclei"
)
# show the modified configuration
print(modified_config["cellprofiler_sqlite"]["CONFIG_JOINS"])


            WITH Per_Image_Filtered AS (
                SELECT
                    Metadata_ImageNumber,
                    Image_Metadata_Well,
                    Image_Metadata_Plate
                FROM
                    read_parquet('per_image.parquet')
                )
            SELECT
                *
            FROM
                Per_Image_Filtered AS per_image
            LEFT JOIN read_parquet('per_cytoplasm.parquet') AS per_cytoplasm ON
                per_cytoplasm.Metadata_ImageNumber = per_image.Metadata_ImageNumber
            LEFT JOIN read_parquet('per_cells.parquet') AS per_cells ON
                per_cells.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_cells.Cells_Number_Object_Number = per_cytoplasm.Cytoplasm_Parent_Cells
            LEFT JOIN read_parquet('per_nuclei.parquet') AS per_nuclei ON
                per_nuclei.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_nuclei.Nuclei_Num

In [11]:
%%time
# pycytominer-transform
# perform merge without annotation and export
# to parquet format, reading the result
# from the parquet file for comparison
pycytominer_transform_sc_df_without_annotation = pd.read_parquet(
    path=convert(
        source_path=str(single_cell_filepath_one),
        dest_path="./pycytominer-transform_singlecells_merge.parquet",
        dest_datatype="parquet",
        merge=True,
        merge_chunk_size=100,
        preset="cellprofiler_sqlite",
        joins=modified_config["cellprofiler_sqlite"]["CONFIG_JOINS"],
    )
)

CPU times: user 3.28 s, sys: 399 ms, total: 3.68 s
Wall time: 3.61 s


In [68]:
pycytominer_transform_sc_df_without_annotation.info()
pycytominer_transform_sc_df_without_annotation.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Columns: 1205 entries, Metadata_ImageNumber to Nuclei_Texture_Variance_RFP_3_03_256
dtypes: float64(1184), int64(19), object(2)
memory usage: 2.2+ MB


,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Well,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Cytoplasm_AreaShape_Center_X,...,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
0,1,001,C6,32065.0,95944.0,1018.0,382.0,750.0,24.0,887.192266,...,1393.410857,1331.583275,653.826838,618.063979,606.832257,590.114791,147.195839,144.355017,148.179465,148.875403
1,1,001,C6,14466.0,58032.0,688.0,337.0,440.0,103.0,591.805060,...,1369.498111,1276.305513,332.941295,317.567450,321.873215,292.116754,60.632767,61.876198,65.202076,60.022847
2,1,001,C6,41368.0,139598.0,1201.0,503.0,888.0,57.0,1048.207286,...,1338.091947,1299.373271,432.829034,398.306003,401.091835,358.849840,74.837374,71.033793,80.523205,80.845266
3,1,001,C6,14564.0,52624.0,377.0,470.0,169.0,217.0,272.484482,...,899.439956,874.837386,211.898029,189.348918,186.333300,188.292692,113.059608,113.194846,110.997393,109.834390
4,1,001,C6,27417.0,84084.0,760.0,550.0,487.0,242.0,632.247620,...,1231.630414,1218.998954,306.139730,295.581509,310.469726,287.788390,496.084704,502.046808,490.259298,491.171009


In [12]:
# check for missing cols from pycytominer to pycytominer-transform
pycytominer_missing_cols = [
    col
    for col in pycytominer_sc_df_without_annotation
    if col not in pycytominer_transform_sc_df_without_annotation.columns
]
pycytominer_missing_cols

['Metadata_Cytoplasm_Parent_Cells',
 'Metadata_Cytoplasm_Parent_Nuclei',
 'Metadata_Cells_Number_Object_Number',
 'Metadata_Nuclei_Number_Object_Number']

In [13]:
# check for missing cols from pycytominer-transform to pycytominer
pt_missing_cols = [
    col
    for col in pycytominer_transform_sc_df_without_annotation.columns
    if col not in pycytominer_sc_df_without_annotation.columns
]
pt_missing_cols

['Cytoplasm_Parent_Cells',
 'Cytoplasm_Parent_Nuclei',
 'Cells_Number_Object_Number',
 'Nuclei_Number_Object_Number']

In [14]:
# form renaming dictionary
rename_for_metadata_prefix = {
    orig: new
    for orig, new in zip(pt_missing_cols, pycytominer_missing_cols)
    if orig in new
}
rename_for_metadata_prefix

{'Cytoplasm_Parent_Cells': 'Metadata_Cytoplasm_Parent_Cells',
 'Cytoplasm_Parent_Nuclei': 'Metadata_Cytoplasm_Parent_Nuclei',
 'Cells_Number_Object_Number': 'Metadata_Cells_Number_Object_Number',
 'Nuclei_Number_Object_Number': 'Metadata_Nuclei_Number_Object_Number'}

In [15]:
# minor rename for existing data
pycytominer_transform_sc_df_without_annotation = (
    pycytominer_transform_sc_df_without_annotation.rename(
        columns=rename_for_metadata_prefix
    )
)

In [16]:
# check for missing cols from pycytominer to pycytominer-transform
pycytominer_missing_cols = [
    col
    for col in pycytominer_sc_df_without_annotation
    if col not in pycytominer_transform_sc_df_without_annotation.columns
]
pycytominer_missing_cols

[]

In [17]:
# check the shape of both dataframes
print(pycytominer_sc_df_without_annotation.shape)
print(pycytominer_transform_sc_df_without_annotation.shape)

(242, 1205)
(242, 1205)


In [18]:
# test dataframe equality
pd.testing.assert_frame_equal(
    left=pycytominer_sc_df_without_annotation,
    right=pycytominer_transform_sc_df_without_annotation[
        # use the pycytominer column order as a reference for pycytominer-transform output
        pycytominer_sc_df_without_annotation.columns
    ],
)

In [ ]:
# Merge single cells across compartments
anno_kwargs = {"join_on": ["Metadata_well_position", "Image_Metadata_Well"]}

sc_df = sc.merge_single_cells(
    platemap=platemap_df,
    **anno_kwargs,
)

# Save level 2 data as a csv
output(sc_df, sc_output_file)

print(sc_df.shape)
sc_df.head()

In [ ]:
# Normalize single cell data and write to file
normalize_sc_df = normalize(sc_df, method="standardize")

output(normalize_sc_df, sc_norm_output_file)

print(normalize_sc_df.shape)
normalize_sc_df.head()

### Visualize basic count statistics

In [ ]:
sc_df.Metadata_genotype.value_counts()

In [ ]:
pd.crosstab(sc_df.Metadata_genotype, sc_df.Metadata_Well)